IMPORT STATEMENT

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
import pickle
import os as os
import tensorflow as tf
from torchvision import transforms
import time
import math
import numpy as np
from settings import *
from data_prep import calc_iou

In [2]:
path = "D:/Semester 6/Deep Learning/uas_dataset/"
train_path = 'output/obj'
test_path = 'test'

FUNCTIONS TO LOAD IMAGE

In [3]:
def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    return image

FUNCTION TO READ .TXT FILE

In [4]:
def load_annotation_from_txt(annotation_path):
    with open(annotation_path, 'r') as f:
        lines = f.readlines()

    annotation = []
    for line in lines:
        line = line.strip().split(' ')
        class_label, xmin, ymin, xmax, ymax= map(float, line)
        annotation.append([class_label, xmin, ymin, xmax, ymax])

    return annotation

LOAD CUSTOM DATASET

In [5]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, annotation_paths, transform=None):
        self.image_paths = image_paths
        self.annotation_paths = annotation_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        # Load the image
        image = load_image(self.image_paths[index])

        # Load and parse the annotation from the .txt file
        annotation = load_annotation_from_txt(self.annotation_paths[index])

        # Preprocess the image and annotation
        if self.transform is not None:
            image, annotation = self.transform(image, annotation)

        return image, annotation

GET ANNOTATION PATH TRAIN


In [6]:
train_txt_files = []
file_list = os.listdir(path + train_path)
for folder in file_list:
    folder_path = os.path.join(path, train_path, folder)
    folder_path = folder_path.replace('\\', '/')
    if os.path.isdir(folder_path):
        # Get a list of all files in the current folder
        folder_files = os.listdir(folder_path)
        # print(folder_files)
        
        # Filter out non-image files based on file extensions
        folder_text_files = [file for file in folder_files if file.lower().endswith((".txt"))]

        # Append the image files of the current folder to the overall image_files list
        for file in folder_text_files:
            file_path = os.path.join(folder_path, file)
            file_path = file_path.replace('\\', '/')
            train_txt_files.extend([file_path])
        
# print(train_txt_files)

GET ANNOTATION PATH TEST

In [7]:
test_txt_files = []
file_list = os.listdir(path + test_path)
for folder in file_list:
    folder_path = os.path.join(path, test_path, folder)
    folder_path = folder_path.replace('\\', '/')
    if os.path.isdir(folder_path):
        # Get a list of all files in the current folder
        folder_files = os.listdir(folder_path)
        # print(folder_files)
        
        # Filter out non-image files based on file extensions
        folder_text_files = [file for file in folder_files if file.lower().endswith((".txt"))]

        # Append the image files of the current folder to the overall image_files list
        for file in folder_text_files:
            file_path = os.path.join(folder_path, file)
            file_path = file_path.replace('\\', '/')
            test_txt_files.extend([file_path])
        

# print(test_txt_files)

GET IMAGE PATH TRAIN

In [8]:
train_image_files = []
file_list = os.listdir(path + train_path)
for folder in file_list:
    folder_path = os.path.join(path, train_path, folder)
    folder_path = folder_path.replace('\\', '/')
    if os.path.isdir(folder_path):
        # Get a list of all files in the current folder
        folder_files = os.listdir(folder_path)
        # print(folder_files)
        
        # Filter out non-image files based on file extensions
        folder_image_files = [file for file in folder_files if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

        # Append the image files of the current folder to the overall image_files list
        for file in folder_image_files:
            file_path = os.path.join(folder_path, file)
            file_path = file_path.replace('\\', '/')
            train_image_files.extend([file_path])   

# print(train_image_files)

GET IMAGE PATH TEST

In [9]:
test_image_files = []
file_list = os.listdir(path + test_path)
for folder in file_list:
    folder_path = os.path.join(path, test_path, folder)
    folder_path = folder_path.replace('\\', '/')
    if os.path.isdir(folder_path):
        # Get a list of all files in the current folder
        folder_files = os.listdir(folder_path)
        # print(folder_files)
        
        # Filter out non-image files based on file extensions
        folder_image_files = [file for file in folder_files if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

        # Append the image files of the current folder to the overall image_files list
        for file in folder_image_files:
            file_path = os.path.join(folder_path, file)
            file_path = file_path.replace('\\', '/')
            test_image_files.extend([file_path])

# print(test_image_files)

DECLARE CUSTOM DATASET

In [10]:
dataset = CustomDataset(train_image_files, train_txt_files)

In [11]:
# import torch
# import torch.nn as nn
# import torchvision.models as models

# class SSDModel(nn.Module):
#     def __init__(self, num_classes):
#         super(SSDModel, self).__init__()

#         # Load the pretrained base model
#         self.base_model = models.vgg16(pretrained=True)

#         # Modify the backbone
#         # ...

#         # Define default box aspect ratios and scales
#         # ...

#         # Define additional layers for prediction
#         # ...

#     def forward(self, x):
#         # Pass the input through the modified backbone
#         features = self.base_model(x)

#         # Generate anchor boxes
#         # ...

#         # Perform prediction on each feature map level
#         # ...

#         # Combine the predictions from different levels
#         # ...

#         return predictions

SSD HOOK FUNC

In [12]:
def SSDHook(feature_map, layer_name):
    """
    SSD hook for generating classification and localization predictions from a feature map
    Args:
        feature_map: Tensor, feature map from a layer in the network
        layer_name: str, name of the layer
    Returns:
        net_conf: Tensor, classification predictions
        net_loc: Tensor, localization predictions
    """
    with tf.name_scope(layer_name):
        # Note we have linear activation (i.e. no activation function)
        net_conf = tf.keras.layers.Conv2D(NUM_PRED_CONF, [3, 3], activation=None, name='conv_conf')(feature_map)
        net_conf = tf.keras.layers.Flatten()(net_conf)

        net_loc = tf.keras.layers.Conv2D(NUM_PRED_LOC, [3, 3], activation=None, name='conv_loc')(feature_map)
        net_loc = tf.keras.layers.Flatten()(net_loc)

    return net_conf, net_loc

MODEL HELPER FUNC

In [13]:
def ModelHelper(y_pred_conf, y_pred_loc):
    """
	Define loss function, optimizer, predictions, and accuracy metric
	Loss includes confidence loss and localization loss
	conf_loss_mask is created at batch generation time, to mask the confidence losses
	It has 1 at locations w/ positives, and 1 at select negative locations
	such that negative-to-positive ratio of NEG_POS_RATIO is satisfied
	Arguments:
		* y_pred_conf: Class predictions from model,
			a tensor of shape [batch_size, num_feature_map_cells * num_default_boxes * num_classes]
		* y_pred_loc: Localization predictions from model,
			a tensor of shape [batch_size, num_feature_map_cells * num_default_boxes * 4]
	Returns relevant tensor references
	"""
    num_total_preds = 0
    for fm_size in FM_SIZES:
        num_total_preds += fm_size[0] * fm_size[1] * NUM_DEFAULT_BOXES
        # print(num_total_preds)
    num_total_preds_conf = num_total_preds * NUM_CLASSES
    num_total_preds_loc = num_total_preds * 4
    print(num_total_preds_loc)
    print(y_pred_loc.shape)

    # Input tensors
    y_true_conf = tf.keras.Input(shape=(num_total_preds,), dtype=tf.int32, name='y_true_conf')
    y_true_loc = tf.keras.Input(shape=(num_total_preds_loc,), dtype=tf.float32, name='y_true_loc')
    conf_loss_mask = tf.keras.Input(shape=(num_total_preds,), dtype=tf.float32, name='conf_loss_mask')

    
    # Confidence loss
    logits = tf.reshape(y_pred_conf, [-1, num_total_preds, NUM_CLASSES])
    conf_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y_true_conf)
    conf_loss = conf_loss_mask * conf_loss
    conf_loss = tf.reduce_sum(conf_loss)
    
    # Localization loss (smooth L1 loss)
    diff = y_true_loc - y_pred_loc

    loc_loss_l2 = 0.5 * (diff ** 2.0)
    loc_loss_l1 = tf.abs(diff) - 0.5
    smooth_l1_condition = tf.less(tf.abs(diff), 1.0)
    loc_loss = tf.where(smooth_l1_condition, loc_loss_l2, loc_loss_l1)

    loc_loss_mask = tf.minimum(y_true_conf, 1)
    loc_loss_mask = tf.cast(loc_loss_mask, tf.float32)
    loc_loss_mask = tf.reshape(loc_loss_mask, [-1, num_total_preds_loc])
    loc_loss = loc_loss_mask * loc_loss
    loc_loss = tf.reduce_sum(loc_loss)

    # Weighted average of confidence loss and localization loss
    # Also add regularization loss
    loss = conf_loss + LOC_LOSS_WEIGHT * loc_loss + tf.reduce_sum(tf.losses.get_regularization_losses())
    optimizer = OPT.minimize(loss)

    probs_all = tf.nn.softmax(logits)
    probs, preds_conf = tf.nn.top_k(probs_all)
    probs = tf.reshape(probs, [-1, num_total_preds])
    preds_conf = tf.reshape(preds_conf, [-1, num_total_preds])

    ret_dict = {
        'y_true_conf': y_true_conf,
        'y_true_loc': y_true_loc,
        'conf_loss_mask': conf_loss_mask,
        'optimizer': optimizer,
        'conf_loss': conf_loss,
        'loc_loss': loc_loss,
        'loss': loss,
        'probs': probs,
        'preds_conf': preds_conf,
        'preds_loc': y_pred_loc,
    }
    return ret_dict

ALEX NET FUNC

In [14]:
def AlexNet():
    """
    AlexNet
    """
    # Image batch tensor and dropout keep prob placeholders
    x = tf.keras.Input(shape=(IMG_H, IMG_W, NUM_CHANNELS), name='x')
    is_training = tf.keras.Input(shape=(), dtype=tf.bool, name='is_training')

    # Classification and localization predictions
    preds_conf = []  # conf -> classification b/c confidence loss -> classification loss
    preds_loc = []

    # Use batch normalization for all convolution layers
    net = tf.keras.layers.BatchNormalization()(x)
    net = tf.keras.layers.Conv2D(64, [3, 3], activation=tf.nn.relu, padding='same', name='conv1_1')(net)
    net = tf.keras.layers.Conv2D(64, [3, 3], activation=tf.nn.relu, padding='same', name='conv1_2')(net)
    net = tf.keras.layers.MaxPooling2D([2, 2], strides=[2, 2], name='pool1')(net)

    net = tf.keras.layers.BatchNormalization()(net)
    net = tf.keras.layers.Conv2D(128, [3, 3], activation=tf.nn.relu, padding='same', name='conv2_1')(net)
    net = tf.keras.layers.Conv2D(128, [3, 3], activation=tf.nn.relu, padding='same', name='conv2_2')(net)
    net = tf.keras.layers.MaxPooling2D([2, 2], strides=[2, 2], name='pool2')(net)

    net = tf.keras.layers.BatchNormalization()(net)
    net = tf.keras.layers.Conv2D(256, [3, 3], activation=tf.nn.relu, padding='same', name='conv3_1')(net)
    net = tf.keras.layers.Conv2D(256, [3, 3], activation=tf.nn.relu, padding='same', name='conv3_2')(net)
    net = tf.keras.layers.Conv2D(256, [3, 3], activation=tf.nn.relu, padding='same', name='conv3_3')(net)
    net = tf.keras.layers.MaxPooling2D([2, 2], strides=[2, 2], name='pool3')(net)

    net = tf.keras.layers.BatchNormalization()(net)
    net = tf.keras.layers.Conv2D(512, [3, 3], activation=tf.nn.relu, padding='same', name='conv4_1')(net)
    net = tf.keras.layers.Conv2D(512, [3, 3], activation=tf.nn.relu, padding='same', name='conv4_2')(net)
    net = tf.keras.layers.Conv2D(512, [3, 3], activation=tf.nn.relu, padding='same', name='conv4_3')(net)
    net = tf.keras.layers.MaxPooling2D([2, 2], strides=[2, 2], name='pool4')(net)

    net = tf.keras.layers.BatchNormalization()(net)
    net = tf.keras.layers.Conv2D(512, [3, 3], activation=tf.nn.relu, padding='same', name='conv5_1')(net)
    net = tf.keras.layers.Conv2D(512, [3, 3], activation=tf.nn.relu, padding='same', name='conv5_2')(net)
    net = tf.keras.layers.Conv2D(512, [3, 3], activation=tf.nn.relu, padding='same', name='conv5_3')(net)

    # The following layers added for SSD
    net = tf.keras.layers.Conv2D(1024, [3, 3], activation=tf.nn.relu, padding='same', name='conv6')(net)
    net = tf.keras.layers.Conv2D(1024, [1, 1], activation=tf.nn.relu, padding='same', name='conv7')(net)

    net_conf, net_loc = SSDHook(net, 'conv7')
    print(net_loc)
    preds_conf.append(net_conf)
    preds_loc.append(net_loc)

    net = tf.keras.layers.Conv2D(256, [1, 1], activation=tf.nn.relu, padding='same', name='conv8')(net)
    net = tf.keras.layers.Conv2D(512, [3, 3], activation=tf.nn.relu, padding='same', name='conv8_2')(net)

    net = tf.keras.layers.Conv2D(256, [1, 1], activation=tf.nn.relu, padding='same', name='conv8')(net)
    net = tf.keras.layers.Conv2D(512, [3, 3], strides=[2, 2], activation=tf.nn.relu, padding='same', name='conv8_2')(net)

    net_conf, net_loc = SSDHook(net, 'conv8_2')
    print(net_loc)
    preds_conf.append(net_conf)
    preds_loc.append(net_loc)

    net = tf.keras.layers.Conv2D(128, [1, 1], activation=tf.nn.relu, padding='same', name='conv9')(net)
    net = tf.keras.layers.Conv2D(256, [3, 3], strides=[2, 2], activation=tf.nn.relu, padding='same', name='conv9_2')(net)

    net_conf, net_loc = SSDHook(net, 'conv9_2')
    print(net_loc)
    preds_conf.append(net_conf)
    preds_loc.append(net_loc)

    # Concatenate all preds together into 1 vector, for both classification and localization predictions
    final_pred_conf = tf.keras.layers.Concatenate(axis=1)(preds_conf)
    final_pred_loc = tf.keras.layers.Concatenate(axis=1)(preds_loc)
    print(final_pred_loc)

    # Return a dictionary of {tensor_name: tensor_reference}
    ret_dict = {
        'x': x,
        'y_pred_conf': final_pred_conf,
        'y_pred_loc': final_pred_loc,
        'is_training': is_training
    }

    return ret_dict

SSD MODEL FUNC

In [15]:
def SSDModel():
    """
	Wrapper around the model and model helper
	Returns dict of relevant tensor references
	"""
    if MODEL == 'AlexNet':
        model = AlexNet()
    else:
        raise NotImplementedError('Model %s not supported' % MODEL)

    model_helper = ModelHelper(model['y_pred_conf'], model['y_pred_loc'])

    ssd_model = {}
    for k in model.keys():
        ssd_model[k] = model[k]
    for k in model_helper.keys():
        ssd_model[k] = model_helper[k]

    return ssd_model

NMS (non maximum supression) FUNC

In [16]:
def nms(y_pred_conf, y_pred_loc, prob):
    class_boxes = {}
    for cls in range(NUM_CLASSES):
        class_boxes[cls] = []

    y_idx = 0
    for fm_size in FM_SIZES:
        fm_h, fm_w = fm_size
        for row in range(fm_h):
            for col in range(fm_w):
                for db in range(NUM_DEFAULT_BOXES):
                    if prob[y_idx] > CONF_THRESH and y_pred_conf[y_idx] > 0.:
                        xc, yc = col + 0.5, row + 0.5
                        center_coords = np.array([xc, yc, xc, yc])
                        abs_box_coords = center_coords + y_pred_loc[y_idx * 4 : y_idx * 4 + 4]

                        scale = np.array([IMG_W / fm_w, IMG_H / fm_h, IMG_W / fm_w, IMG_H / fm_h])
                        box_coords = abs_box_coords * scale
                        box_coords = [int(round(x)) for x in box_coords]

                        cls = y_pred_conf[y_idx]
                        cls_prob = prob[y_idx]
                        box = (*box_coords, cls, cls_prob)
                        if len(class_boxes[cls]) == 0:
                            class_boxes[cls].append(box)
                        else:
                            suppressed = False
                            overlapped = False
                            for other_box in class_boxes[cls]:
                                iou = calc_iou(box[:4], other_box[:4])
                                if iou > NMS_IOU_THRESH:
                                    overlapped = True
                                    if box[5] > other_box[5]:
                                        class_boxes[cls].remove(other_box)
                                        suppressed = True
                            if suppressed or not overlapped:
                                class_boxes[cls].append(box)

                    y_idx += 1

    boxes = []
    for cls in class_boxes.keys():
        for class_box in class_boxes[cls]:
            boxes.append(class_box)
    boxes = np.array(boxes)

    return boxes

NEXT BATCH FUNC

In [17]:
def next_batch(X, y_conf, y_loc, batch_size):
	"""
	Next batch generator
	Arguments:
		* X: List of image file names
		* y_conf: List of ground-truth vectors for class labels
		* y_loc: List of ground-truth vectors for localization
		* batch_size: Batch size

	Yields:
		* images: Batch numpy array representation of batch of images
		* y_true_conf: Batch numpy array of ground-truth class labels
		* y_true_loc: Batch numpy array of ground-truth localization
		* conf_loss_mask: Loss mask for confidence loss, to set NEG_POS_RATIO
	"""
	transform = transforms.Compose([
		transforms.Resize((IMG_W, IMG_H)),
		transforms.ToTensor(),
	# Add other transformations as needed
	])

	train_dataset = CustomDataset(X, y_conf, y_loc, transform=transform)
	train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

	for images, y_true_conf, y_true_loc in train_dataloader:
		# Grayscale images have shape (batch_size, H, W), but we want shape (batch_size, H, W, 1)
		if NUM_CHANNELS == 1:
			images = images.unsqueeze(3)
		elif NUM_CHANNELS == 3:
			images = images.unsqueeze(3).expand(-1, -1, -1, 3)

		# Normalize pixel values (scale them between -1 and 1)
		images = images / 127.5 - 1.

		# For y_true_conf, calculate how many negative examples we need to satisfy NEG_POS_RATIO
		num_pos = torch.sum(y_true_conf > 0)
		num_neg = NEG_POS_RATIO * num_pos
		y_true_conf_size = y_true_conf.numel()

		# Create confidence loss mask to satisfy NEG_POS_RATIO
		if num_pos + num_neg < y_true_conf_size:
			conf_loss_mask = y_true_conf.clone()
			conf_loss_mask[y_true_conf == 1] = 1
			num_neg_needed = y_true_conf_size - num_pos
			neg_indices = torch.where(y_true_conf == 0)[0]
			random_indices = torch.randperm(neg_indices.size(0))[:num_neg_needed]
			neg_indices_to_keep = neg_indices[random_indices]
			conf_loss_mask[neg_indices_to_keep] = 1
		else:
			conf_loss_mask = torch.ones_like(y_true_conf)

		yield images, y_true_conf, y_true_loc, conf_loss_mask

RUN TRAINING FUNC

In [18]:
def run_training():
    """
    Load training and test data
    Run training process
    Plot train/validation losses
    Report test loss
    Save model
    """
    # Load training and test data
    train_image_paths = train_image_files  # List of image file paths
    train_annotation_paths = train_txt_files  # List of annotation file paths

    test_image_paths = test_image_files  # List of image file paths
    test_annotation_paths = test_txt_files  # List of annotation file paths
    
    transform = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Resizing(IMG_W, IMG_H),
        tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
        # Add other transformations as needed
    ])

    train_dataset = CustomDataset(train_image_paths, train_annotation_paths, transform=transform)
    X_train = train_dataset.image_paths
    y_train_conf = train_dataset.annotation_paths
    y_train_loc = train_dataset.annotation_paths

    X_train = np.array(X_train)
    y_train_conf = np.array(y_train_conf)
    y_train_loc = np.array(y_train_loc)

    test_dataset = CustomDataset(test_image_paths, test_annotation_paths, transform=transform)
    X_valid = test_dataset.image_paths
    y_valid_conf = test_dataset.annotation_paths
    y_valid_loc = test_dataset.annotation_paths

    X_valid = np.array(X_valid)
    y_valid_conf = np.array(y_valid_conf)
    y_valid_loc = np.array(y_valid_loc)
    
    # Instantiate neural network, get relevant tensors
    model = SSDModel()
    x = model['x']
    y_true_conf = model['y_true_conf']
    y_true_loc = model['y_true_loc']
    conf_loss_mask = model['conf_loss_mask']
    is_training = model['is_training']
    optimizer = model['optimizer']
    reported_loss = model['loss']

    # Training process
    # TF saver to save/restore trained model
    saver = tf.train.Checkpoint(model=model)
    manager = tf.train.CheckpointManager(saver, './model', max_to_keep=1)

    if RESUME:
        print('Restoring previously trained model')
        checkpoint.restore(manager.latest_checkpoint)
        
        # Restore previous loss history
        with open('loss_history.p', 'rb') as f:
            loss_history = pickle.load(f)
    else:
        print('Training model from scratch')
        # Variable initialization
        model.initialize()

        # For book-keeping, keep track of training and validation loss over epochs
        loss_history = []

    # Record time elapsed for performance check
    last_time = time.time()
    train_start_time = time.time()

    # Run NUM_EPOCH epochs of training
    for epoch in range(NUM_EPOCH):
        train_gen = next_batch(X_train, y_train_conf, y_train_loc, BATCH_SIZE)
        num_batches_train = math.ceil(X_train.shape[0] / BATCH_SIZE)
        losses = []  # list of loss values for book-keeping

        # Run training on each batch
        for _ in range(num_batches_train):
            # Obtain the training data and labels from generator
            images, y_true_conf_gen, y_true_loc_gen, conf_loss_mask_gen = next(train_gen)

            # Perform gradient update (i.e. training step) on current batch
            with tf.GradientTape() as tape:
                loss = model.compute_loss(images, y_true_conf_gen, y_true_loc_gen, conf_loss_mask_gen)
            
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            
            losses.append(loss)  # TODO: Need mAP metric instead of raw loss

        # A rough estimate of loss for this epoch (overweights the last batch)
        train_loss = np.mean(losses)

        # Calculate validation loss at the end of the epoch
        valid_gen = next_batch(X_valid, y_valid_conf, y_valid_loc, BATCH_SIZE)
        num_batches_valid = math.ceil(X_valid.shape[0] / BATCH_SIZE)
        losses = []
        for _ in range(num_batches_valid):
            images, y_true_conf_gen, y_true_loc_gen, conf_loss_mask_gen = next(valid_gen)

            # Perform forward pass and calculate loss
            loss = model.compute_loss(images, y_true_conf_gen, y_true_loc_gen, conf_loss_mask_gen)
            losses.append(loss)
        valid_loss = np.mean(losses)

        # Record and report train/validation/test losses for this epoch
        loss_history.append((train_loss, valid_loss))

        # Print accuracy every epoch
        print('Epoch %d -- Train loss: %.4f, Validation loss: %.4f, Elapsed time: %.2f sec' %\
            (epoch+1, train_loss, valid_loss, time.time() - last_time))
        last_time = time.time()

    total_time = time.time() - train_start_time
    print('Total elapsed time: %d min %d sec' % (total_time/60, total_time%60))

    test_loss = 0.  # TODO: Add test set
    '''
    # After training is complete, evaluate accuracy on test set
    print('Calculating test accuracy...')
    test_gen = next_batch(X_test, y_test, BATCH_SIZE)
    test_size = X_test.shape[0]
    test_acc = calculate_accuracy(test_gen, test_size, BATCH_SIZE, accuracy, x, y, keep_prob, sess)
    print('Test acc.: %.4f' % (test_acc,))
    '''

    if SAVE_MODEL:
        # Save model to disk
        save_path = manager.save()
        print('Trained model saved at: %s' % save_path)

        # Also save accuracy history
        print('Loss history saved at loss_history.p')
        with open('loss_history.p', 'wb') as f:
            pickle.dump(loss_history, f)

    # Return final test accuracy and accuracy_history
    return test_loss, loss_history

MAIN

In [19]:
if __name__ == '__main__':
	run_training()

KerasTensor(type_spec=TensorSpec(shape=(None, 5152), dtype=tf.float32, name=None), name='flatten_1/Reshape:0', description="created by layer 'flatten_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 1056), dtype=tf.float32, name=None), name='flatten_3/Reshape:0', description="created by layer 'flatten_3'")
KerasTensor(type_spec=TensorSpec(shape=(None, 160), dtype=tf.float32, name=None), name='flatten_5/Reshape:0', description="created by layer 'flatten_5'")
KerasTensor(type_spec=TensorSpec(shape=(None, 6368), dtype=tf.float32, name=None), name='concatenate_1/concat:0', description="created by layer 'concatenate_1'")
31248
(None, 6368)


ValueError: Exception encountered when calling layer "tf.math.subtract" (type TFOpLambda).

Dimensions must be equal, but are 31248 and 6368 for '{{node tf.math.subtract/Sub}} = Sub[T=DT_FLOAT](Placeholder, Placeholder_1)' with input shapes: [?,31248], [?,6368].

Call arguments received by layer "tf.math.subtract" (type TFOpLambda):
  • x=tf.Tensor(shape=(None, 31248), dtype=float32)
  • y=tf.Tensor(shape=(None, 6368), dtype=float32)
  • name=None